# Iris classification in Keras

Author: Michał Słapek

Classification example for Iris dataset.

In [ ]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
import seaborn as sns

import keras
from keras.models import Sequential
from keras.layers import Dense

from utils import get_iris_data
from plot_iris import plot_contours

In [ ]:
data = get_iris_data()

In [ ]:
type(data)

In [ ]:
data.head(5)

In [ ]:
sns.set()
plt.figure(dpi=100)
sns.scatterplot(
    x='sepal length (cm)',
    y='sepal width (cm)',
    hue='family',
    data=data
)
plt.show()

## Model plot

In [ ]:
plot_x_lim = (4, 9)
plot_y_lim = (1.5, 5)
plot_X, plot_Y = np.meshgrid(
    np.arange(*plot_x_lim, step=0.1),
    np.arange(*plot_y_lim, step=0.1)
)

def format_loss(l):
    return '[' + ', '.join(f'{v:0.2f}' for v in l) + ']'

def plot_model(model, name='model'):
    fig, ax = plt.subplots(dpi=100)
    a = plot_contours(ax, model, plot_X, plot_Y)
    fig.colorbar(a)
    sns.scatterplot(
        x=X_train[:, 0],
        y=X_train[:, 1],
        hue='T' + pd.Series(y_train).astype(str),
        hue_order=[f'T{i}' for i in range(3)],
        marker='x',
        legend=False
    )
    sns.scatterplot(
        x=X_valid[:, 0],
        y=X_valid[:, 1],
        hue='V' + pd.Series(y_valid).astype(str),
        hue_order=[f'V{i}' for i in range(3)],
        legend=False
    )
    
    text = f'train: {format_loss(model.evaluate(X_train, y_train, verbose=False))}\n' \
           f'valid: {format_loss(model.evaluate(X_valid, y_valid, verbose=False))}'
    ax.text(4.5, 4.5, text, color='white')

## Train, test, validation split

In [ ]:
from sklearn.model_selection import train_test_split

X = data[['sepal length (cm)', 'sepal width (cm)']].values
y = data['class'].values

X_train_valid, X_test, y_train_valid, y_test = train_test_split(
    X, y,
    test_size=0.25,
    random_state=910797
)

X_train, X_valid, y_train, y_valid = train_test_split(
    X_train_valid, y_train_valid,
    test_size=1/3,
    random_state=142385
)

for m in [X_train, X_valid, X_test]:
    print(type(m))
    print(m.shape)    

## Model class ~ (sepal width) * (sepal length)

In [ ]:
models = {}

### Flat model

In [ ]:
model = Sequential()
model.add(Dense(units=3, input_dim=2, activation='softmax'))

model.compile(
    loss='sparse_categorical_crossentropy',
    optimizer=keras.optimizers.SGD(lr=0.01, momentum=0.9, nesterov=True),
    metrics=['accuracy']
)
model.summary()

In [ ]:
# model.load_weights('weights/iris_flat.hdf5')
model.fit(
    X_train, y_train, 
    epochs=1_000, 
    batch_size=len(y_train),
    validation_data=(X_valid, y_valid)
)
# model.save_weights('weights/iris_flat.hdf5')

In [ ]:
models['flat'] = model
plot_model(model, 'flat')

## Medium model

In [ ]:
model = Sequential()
model.add(Dense(units=5, input_dim=2, activation='relu'))
model.add(Dense(units=3, activation='softmax'))

model.compile(
    loss='sparse_categorical_crossentropy',
    optimizer=keras.optimizers.SGD(lr=0.01, momentum=0.9, nesterov=True),
    metrics=['accuracy']
)
model.summary()

In [ ]:
model.load_weights('weights/iris_medium.hdf5')
# model.fit(
#     X_train, y_train, 
#     epochs=10_000, 
#     batch_size=len(y_train),
#     validation_data=(X_valid, y_valid),
#     verbose=False
# )
# model.save_weights('weights/iris_medium.hdf5')

In [ ]:
models['medium'] = model
plot_model(model)

## Big model

In [ ]:
model = Sequential()
model.add(Dense(units=100, input_dim=2, activation='relu'))
model.add(Dense(units=100, activation='relu'))
model.add(Dense(units=100, activation='relu'))
model.add(Dense(units=3, activation='softmax'))

model.compile(
    loss='sparse_categorical_crossentropy',
    optimizer=keras.optimizers.SGD(lr=0.01, momentum=0.9, nesterov=True),
    metrics=['accuracy']
)
model.summary()

In [ ]:
model.load_weights('weights/iris_big.hdf5')
# model.fit(
#     X_train, y_train, 
#     epochs=10_000, 
#     batch_size=len(y_train),
#     validation_data=(X_valid, y_valid),
#     verbose=False
# )
# model.save_weights('weights/iris_big.hdf5')

In [ ]:
models['big'] = model
plot_model(model)

In [ ]:
for k, m in models.items():
    score = m.evaluate(X_valid, y_valid, verbose=False)
    print(f'Model {k:10}: {score}')

In [ ]:
models['????'].evaluate(X_test, y_test)